<a target="_blank" href="https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/integrations/callbacks/uptrain.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# UpTrain

> UpTrain [[github](https://github.com/uptrain-ai/uptrain) || [website](https://uptrain.ai/) || [docs](https://docs.uptrain.ai/getting-started/introduction)] is an open-source platform to evaluate and improve LLM applications. It provides grades for 20+ preconfigured checks (covering language, code, embedding use cases), performs root cause analyses on instances of failure cases and provides guidance for resolving them.

## UpTrain Callback Handler

This notebook showcases the UpTrain callback handler seamlessly integrating into your pipeline, facilitating diverse evaluations. We have chosen a few evaluations that we deemed apt for evaluating the chains. These evaluations run automatically, with results displayed in the output. More details on UpTrain's evaluations can be found [here](https://github.com/uptrain-ai/uptrain?tab=readme-ov-file#pre-built-evaluations-we-offer-). 

Selected retievers from Langchain are highlighted for demonstration:

## 1. **RAG Query Engine Evaluations**:
The RAG query engine plays a crucial role in retrieving context and generating responses. To ensure its performance and response quality, we conduct the following evaluations:

- **Context Relevance**: Determines if the context extracted from the query is relevant to the response.
- **Factual Accuracy**: Assesses if the LLM is hallcuinating or providing incorrect information.
- **Response Completeness**: Checks if the response contains all the information requested by the query.

## 2. **Multi Query Generation Evaluation**:
MultiQueryRetriever creates multiple variants of a question having a similar meaning to the original question. Given the complexity, we include the previous evaluations and add:

- **Multi Query Accuracy**: Assures that the multi-queries generated mean the same as the original query.

## 3. **Re-Ranking Evaluations**:
Re-ranking involves reordering nodes based on relevance to the query and choosing top n nodes. Since the number of nodes can reduce once the re-ranking is complete, we perform the following evaluations:

- **Context Reranking**: Checks if the order of re-ranked nodes is more relevant to the query than the original order.
- **Context Conciseness**: Examines whether the reduced number of nodes still provides all the required information.

These evaluations collectively ensure the robustness and effectiveness of the RAG query engine, MultiQueryRetriever, and the re-ranking process in the chain.

## Install Dependencies

In [1]:
# %pip install -qU uptrain langchain faiss flashrank

## Import Libraries

In [2]:
from langchain_community.document_loaders import TextLoader, WebBaseLoader
from langchain_community.vectorstores import FAISS, Chroma
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

from langchain.callbacks.uptrain_callback import UpTrainCallbackHandler
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.retrievers.multi_query import MultiQueryRetriever

/Users/dhruvchawla/work/langchain/.venv/lib/python3.11/site-packages/lazy_loader/__init__.py:185: RuntimeWarning: subpackages can technically be lazily loaded, but it causes the package to be eagerly loaded even if it is already lazily loaded.So, you probably shouldn't use subpackages with this lazy feature.
  warnings.warn(msg, RuntimeWarning)


## Load the documents

In [3]:
loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()

## Split the document into chunks

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = text_splitter.split_documents(documents)

## Create the retriever

In [5]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(chunks, embeddings)
retriever = db.as_retriever()

## Define the LLM

In [6]:
llm = ChatOpenAI(temperature=0, model="gpt-4")

# 1. Vanilla RAG

UpTrain callback handler will automatically capture the query, context and response once generated and will run the following three evaluations *(Graded from 0 to 1)* on the response:
- **Context Relevance**: Check if the context extractedfrom the query is relevant to the response.
- **Factual Accuracy**: Check how factually accurate the response is.
- **Response Completeness**: Check if the response contains all the information that the query is asking for.

In [7]:
# Create the RAG prompt
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
rag_prompt_text = ChatPromptTemplate.from_template(template)

# Create the chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt_text
    | llm
    | StrOutputParser()
)

# Create the uptrain callback handler
uptrain_callback = UpTrainCallbackHandler()
config = {"callbacks": [uptrain_callback]}

# Invoke the chain with a query
query = "What did the president say about Ketanji Brown Jackson"
docs = chain.invoke(query, config=config)

2024-04-03 19:03:22.361 | INFO     | uptrain.framework.evalllm:evaluate_on_server:376 - Sending evaluation request for rows 0 to <50 to the Uptrain
2024-04-03 19:03:43.670 | INFO     | uptrain.framework.evalllm:evaluate:365 - Local server not running, start the server to log data and visualize in the dashboard!



Question: What did the president say about Ketanji Brown Jackson
Response: The president mentioned that he had nominated Ketanji Brown Jackson to serve on the United States Supreme Court 4 days ago. He described her as one of the nation's top legal minds who will continue Justice Breyer’s legacy of excellence. He also mentioned that she is a former top litigator in private practice, a former federal public defender, and comes from a family of public school educators and police officers. He described her as a consensus builder and noted that since her nomination, she has received a broad range of support from various groups including the Fraternal Order of Police and former judges appointed by both Democrats and Republicans.

Context Relevance Score: 0.9
Factual Accuracy Score: 1.0
Response Completeness Score: 0.3



# 2. Multi Query RAG Evaluation

The **MultiQueryRetriever** is used to tackle the problem that the RAG pipeline might not return the best set of documents based on the query. It generates multiple queries that mean the same as the original query and then fetches documents for each.

To evluate this retriever, UpTrain will run the following evaluation:
- **Multi Query Accuracy**: Checks if the multi-queries generated mean the same as the original query.

In [8]:
# Create the retriever
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)

# Create the uptrain callback
uptrain_callback = UpTrainCallbackHandler()
config = {"callbacks": [uptrain_callback]}

# Create the RAG prompt
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
rag_prompt_text = ChatPromptTemplate.from_template(template)

chain = (
    {"context": multi_query_retriever, "question": RunnablePassthrough()}
    | rag_prompt_text
    | llm
    | StrOutputParser()
)

# Invoke the chain with a query
question = "What did the president say about Ketanji Brown Jackson"
# unique_docs = multi_query_retriever.get_relevant_documents(query=question, callbacks=[uptrain_callback])
docs = chain.invoke(question, config=config)

2024-04-03 19:03:52.670 | INFO     | uptrain.framework.evalllm:evaluate_on_server:376 - Sending evaluation request for rows 0 to <50 to the Uptrain
2024-04-03 19:03:58.738 | INFO     | uptrain.framework.evalllm:evaluate:365 - Local server not running, start the server to log data and visualize in the dashboard!



Question: What did the president say about Ketanji Brown Jackson
Multi Queries:
  How did the president comment on Ketanji Brown Jackson?
  What were the president's remarks regarding Ketanji Brown Jackson?
  Can you provide the president's statements about Ketanji Brown Jackson?

Multi Query Accuracy Score: 1.0



2024-04-03 19:04:09.669 | INFO     | uptrain.framework.evalllm:evaluate_on_server:376 - Sending evaluation request for rows 0 to <50 to the Uptrain
2024-04-03 19:04:41.489 | INFO     | uptrain.framework.evalllm:evaluate:365 - Local server not running, start the server to log data and visualize in the dashboard!



Question: What did the president say about Ketanji Brown Jackson
Response: The president mentioned that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court 4 days ago. He described her as one of the nation's top legal minds who will continue Justice Breyer’s legacy of excellence. He also mentioned that she is a former top litigator in private practice, a former federal public defender, and comes from a family of public school educators and police officers. Since her nomination, she has received a broad range of support, including from the Fraternal Order of Police and former judges appointed by both Democrats and Republicans.

Context Relevance Score: 1.0
Factual Accuracy Score: 1.0
Response Completeness Score: 0.3



# 3. Context Re-Ranking

The re-ranking process involves reordering nodes based on relevance to the query and choosing the top n nodes. Since the number of nodes can reduce once the re-ranking is complete, we perform the following evaluations:
- **Context Reranking**: Check if the order of re-ranked nodes is more relevant to the query than the original order.
- **Context Conciseness**: Check if the reduced number of nodes still provides all the required information.

In [9]:
# Create the retriever
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# Create the chain
chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

# Create the uptrain callback
uptrain_callback = UpTrainCallbackHandler()
config = {"callbacks": [uptrain_callback]}

# Invoke the chain with a query
query = "What did the president say about Ketanji Brown Jackson"
result = chain.invoke(query, config=config)

2024-04-03 19:04:44.148 | INFO     | uptrain.framework.evalllm:evaluate_on_server:376 - Sending evaluation request for rows 0 to <50 to the Uptrain
2024-04-03 19:04:54.087 | INFO     | uptrain.framework.evalllm:evaluate:365 - Local server not running, start the server to log data and visualize in the dashboard!



Question: What did the president say about Ketanji Brown Jackson

Context Conciseness Score: 0.0
Context Reranking Score: 1.0



2024-04-03 19:04:59.937 | INFO     | uptrain.framework.evalllm:evaluate_on_server:376 - Sending evaluation request for rows 0 to <50 to the Uptrain
2024-04-03 19:05:21.232 | INFO     | uptrain.framework.evalllm:evaluate:365 - Local server not running, start the server to log data and visualize in the dashboard!



Question: What did the president say about Ketanji Brown Jackson
Response: The President mentioned that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court 4 days ago. He described her as one of the nation's top legal minds who will continue Justice Breyer’s legacy of excellence.

Context Relevance Score: 1.0
Factual Accuracy Score: 1.0
Response Completeness Score: 0.4

